# Integrantes do grupo:
# Andreza Lukosiunas - @andrezasp (nusp: 7157922)
# Juliano Garcia - @robotenique (nusp: 9277086)
# Pedro Carvalho - @pHrfo (nusp: 11376164)

In [ ]:
# Pacotes utilizados

# Conectar-se ao Drive
from google.colab import drive

# https://docs.python.org/3/library/os.html
import os

# https://matplotlib.org/
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# https://pandas.pydata.org/
import pandas as pd

# https://numpy.org/
import numpy as np

# https://docs.python.org/3/library/itertools.html
import itertools

# https://pypi.org/project/Pillow/
from PIL import Image

# skimage para carregar as imagens
# https://scikit-image.org/
import skimage
from skimage import data
from skimage.morphology import disk, ball
from skimage.filters.rank import gradient, median
from skimage import filters
# https://scikit-image.org/docs/dev/api/skimage.io.html
from skimage import io

# https://pypi.org/project/opencv-python/
import cv2

# https://pypi.org/project/tqdm/
# from tqdm import tqdm
from tqdm.notebook import tqdm

# https://docs.python.org/3/library/shutil.html
import shutil

# https://docs.python.org/3/library/pathlib.html
from pathlib import Path

# https://docs.python.org/3/library/glob.html
import glob

# https://docs.python.org/3/library/collections.html
from collections import Counter

# https://docs.python.org/3/library/math.html
import math

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Diretório do trabalho no drive

In [ ]:
os.chdir('/content/gdrive/My Drive/MAC5768 - Visão computacional/')

# EP2
## Nas primeiras células temos as declarações das funções
## Nas últimas, a parte que as usa em: Main - Roda tudo!

## EP2 - Parte 1 | Criação do originalGrayDataset e augmentedDataset

## Funções de filtros e normalização

In [ ]:
# Dict para atribuir nome do folder a id. Utilizado para salvar a imagem no folder, o nome do arquivo conterá o key do dict
folder_dict = {
    **dict.fromkeys(['gray2'], 'originalGrayDataset'), 
    **dict.fromkeys(['gray', 'excgrad', 'grad', 'log', 'exp', 'mean'], 'augmentedDataset'),
    **dict.fromkeys(['equ'], 'normalizedDataset')
}

def resize_image(img):
  height = int(img.shape[0] * 0.2)
  width = int(img.shape[1] * 0.2)
  im_resized = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_CUBIC)
  return im_resized

def save_image(image, file, id_folder):
    folder = folder_dict[id_folder] + "/"
    new_file = file.split(".")[0] + "_" + id_folder + "." + file.split(".")[1]
    cv2.imwrite(folder + new_file, image)
    new_metadados.append({'arquivo': file, 'convertida': new_file})
    return True
    
# Função 1: RGB2gray (ie converter as imagens RGB originais em níveis de cinza)
# https://docs.opencv.org/3.4/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab  
def grayscale_correction(image, file):
    grayscale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    save_image(grayscale, file, 'gray2')
    save_image(grayscale, file, 'gray')
    return grayscale
    
# Função 2: Calcula gradiente de níveis de cinza da imagem, através da utilização de um filtro gaussiano
# com sigma razoavelmente alto, e multiplica a imagem original pelo inverso do gradiente para obter
# uma imagem com correção na sombra. O sigma utilizado usa uma proporção com base nos exemplos do livro
# Digital Image Processing do Woods como heurística para escolher o valor.
def gradient_shade_correction(image, file):
    gauss_sigma = (max(image.shape)*512)/2048
    img_gauss = filters.gaussian(image, sigma=gauss_sigma)
    corrected_image = image/img_gauss
    # normaliza a imagem do gradiente para salvar sem perder informação
    norm_image = cv2.normalize(img_gauss, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    norm_image = norm_image.astype(np.uint8)
    save_image(norm_image, file, 'grad')
    save_image(corrected_image, file, 'excgrad')
    return corrected_image

# Função 3: Converte a imagem original para o logaritmo da imagem
# Essa função transforam os pixels conforme a equação O = gain*log(1 + I) depois os escala entre 0 e 1
# https://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.adjust_log
def log_correction(image, file):
    logarithmic_corrected = skimage.exposure.adjust_log(image, 1)
    save_image(logarithmic_corrected, file, 'log')
    return logarithmic_corrected

# Função 4: Converte a imagem original para o exponencial da imagem
# Essa função transforam os pixels conforme a equação O = I**gamma depois os escala entre 0 e 1
# https://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.adjust_gamma
def exp_correction(image, file):
    exponential_corrected = skimage.exposure.adjust_gamma(image, 1)
    save_image(exponential_corrected, file, 'exp')
    return exponential_corrected

# Função 5: Converte a imagem original para o filtro da média com convolução - usando cv2
# https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04
def mean_filter_convolution(image, file, kernel_size):
    kernel = np.ones((kernel_size,kernel_size),np.float32)/(kernel_size * kernel_size)
    mean_filter_conv = cv2.filter2D(image, -1, kernel)
    save_image(mean_filter_conv, file, 'mean')
    return mean_filter_conv

# Função 6: Normalização das imagens através de equalização
# https://docs.opencv.org/master/d5/daf/tutorial_py_histogram_equalization.html
def equalizer_hist(image, file):
    equalized = cv2.equalizeHist(image)
    save_image(equalized, file, 'equ')
    return equalized

## Função deleta folder, caso necessário, e cria os originalGrayDataset, augmentedDataset e normalizedDataset

In [ ]:
# Deleta e cria o folder novamente originalGrayDataset, augmentedDataset e normalizedDataset
def new_folder(run=False):
  if run:
    img_folders = ['originalGrayDataset', 'augmentedDataset', 'normalizedDataset']
    print('Os folders serão deletados!')
    for folder in img_folders:
      try:
        shutil.rmtree(folder)
        Path(folder).mkdir(parents=True, exist_ok=True)
      except:
        pass 

## Funções para gerar as imagens com todos os 5 filtros e salvá-las em originalGrayDataset e augmentedDataset, e para plotagem da classe escolhida

In [ ]:
# Salva imagens convertidas pelas funções, também salva o nome da imagem em new_metadados
def create_img_filtered(run=False):
  if run:
    global new_metadados
    new_metadados = []
    for file in tqdm(files, position=0):
        img = mpimg.imread("dados/" + file)
        # Resize a imagem para eficiência de tempo
        img = resize_image(img)
        # salva em originalGrayDataset
        grayscale = grayscale_correction(img, file)
        # salva em augmentedDataset
        corrected_image = gradient_shade_correction(grayscale, file)
        logarithmic_corrected = log_correction(grayscale, file)
        exponential_corrected = exp_correction(grayscale, file)
        mean_filter_conv = mean_filter_convolution(grayscale, file, 100)

    # Metadados carregado nas funções acima
    new_metadados_1 = new_metadados.copy()
    new_metadados_11 = {k: [dic[k] for dic in new_metadados_1] for k in new_metadados_1[0]}

    # salva o novo metadatos
    new_metadados_11 = pd.DataFrame(new_metadados_11)
    new_metadados_11_gray = new_metadados_11[new_metadados_11['convertida'].str.contains("_gray2.")]
    new_metadados_11_aug = new_metadados_11[~new_metadados_11['convertida'].str.contains("_gray2.")]
    new_metadados_11_gray.to_csv('metadados/metadados_originalGrayDataset.csv')
    new_metadados_11_aug.to_csv('metadados/metadados_augmentedDataset.csv')


# Auxiliar para a escolher uma classe
def let_user_pick(options):
    print("Qual classe você quer visualizar?")
    for idx, element in enumerate(options):
        print("{}) {}".format(idx+1,element))
    i = input("Coloque o número: ")
    try:
        if 0 < int(i) <= len(options):
            return int(i)
    except:
        pass
    return None


# Plota algumas imagens dos métodos
def plot_img_filtered(run=False):
  if run:
    plot_img = []
    label_img = []
    files_plot = metadados2[metadados2.repeticao == 1]
    classes = files_plot.classe.unique()
    dict_folders = {'originalGrayDataset': 'gray', 'augmentedDataset': 'convertida', 'normalizedDataset': 'normalizada'}

    # Escolha da classe
    options = metadados.classe.unique()
    class_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    class_dict = dict(zip(class_id, options))
    answer = let_user_pick(options)
    answer = class_dict[answer]

    for folder in dict_folders:
      files_plot_aux = files_plot[files_plot.classe == answer]
      for idx, file in enumerate(set(files_plot_aux[dict_folders[folder]])):
          plot_img.append(mpimg.imread(folder + "/" + file))
          label_img.append(file)  
          if idx > 12:
            break

    num_col = 6
    num_row = math.ceil(len(plot_img)/num_col)
    i = 0
    j = 0

    fig, axes = plt.subplots(num_row, num_col, figsize=(50, 30))
    for img, label in zip(plot_img, label_img):
      if num_row == 1:
        ax = axes[j]
      else:  
        ax = axes[i, j]
      ax.imshow(img, aspect='equal', cmap=plt.cm.gray)
      ax.set_title(label, fontsize=20)
      ax.set_axis_off()
      j = j + 1
      if j == num_col:
        j = 0
        i = i + 1

    # fig.subplots_adjust(wspace=0, hspace=0)
    fig.show()

## Função para normalização com equalizador, embora seja componente da parte 2, deixamos aqui para que pudesse ser plotada junto com o augumened

In [ ]:
# Salva imagens normalizadas, também salva o nome da imagem em new_metadados
def create_img_normalized(run=False):
  if run:
    global new_metadados
    new_metadados = []
    # Lê dados do augmentedDataset
    new_metadados_11_aug = pd.read_csv('metadados/metadados_augmentedDataset.csv')
    list_img_name = set(list(new_metadados_11_aug['convertida']))

    for file in tqdm(list_img_name, position=0):
        # considera somente se estiver em augmentedDataset
        try:
            img = mpimg.imread("augmentedDataset/" + file)
            equalizer = equalizer_hist(img, file)
        except:
            pass

    # Metadados carregado nas funções acima    
    new_metadados_2 = new_metadados.copy()
    new_metadados_21 = {k: [dic[k] for dic in new_metadados_2] for k in new_metadados_2[0]}
    new_metadados_21['normalizada'] = new_metadados_21.pop('convertida')
    new_metadados_21['convertida'] = new_metadados_21.pop('arquivo')

    # salva o novo metadatos
    new_metadados_21 = pd.DataFrame(new_metadados_21)
    new_metadados_21.to_csv('metadados/metadados_normalizedDataset.csv')

# Main - Roda tudo!

## Metadados Inicial

In [ ]:
metadados = pd.read_csv('metadados/metadados.csv', encoding='utf-8', delimiter=",", index_col=0)

In [ ]:
metadados.head()

,arquivo,classe,objecto,repeticao,fundo,iluminacao,local,resolucao,tamanho
0,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0
1,20210417_153014(0).jpg,óculos,óculos gucci,2,branco_2,dia,interno,2560x1440,1128321.0
2,20210417_153014.jpg,óculos,óculos gucci,3,branco_2,dia,interno,2560x1440,1118719.0
3,20210417_153025(0).jpg,óculos,óculos dior,1,branco_2,dia,interno,2560x1440,1084526.0
4,20210417_153025.jpg,óculos,óculos dior,2,branco_2,dia,interno,2560x1440,1135350.0


In [ ]:
# Cria lista com os arquivos que vamos trabalhar
files = metadados['arquivo'].values
#files = files[:100]

In [ ]:
# Essas já foram rodadas, não há necessidade de rodar novamente
# Deleta folders e cria novamente: originalGrayDataset, augmentedDataset e normalizedDataset
run_all = False
new_folder(run=run_all)
# Cria imagens em originalGrayDataset e augmentedDataset (leva cerca de 12 horas)
create_img_filtered(run=run_all)
# Cria imagens em normalizedDataset
create_img_normalized(run=run_all)

Os folders serão deletados!


## Carrega os novos metadados e faz o merge

In [ ]:
# Carrega os dados
metadados_originalGrayDataset = pd.read_csv('metadados/metadados_originalGrayDataset.csv', encoding='utf-8', delimiter=",", index_col=0)
metadados_augmentedDataset = pd.read_csv('metadados/metadados_augmentedDataset.csv', encoding='utf-8', delimiter=",", index_col=0)
metadados_normalizedDataset = pd.read_csv('metadados/metadados_normalizedDataset.csv', encoding='utf-8', delimiter=",", index_col=0)
metadados_originalGrayDataset = metadados_originalGrayDataset.rename(columns={'convertida': 'gray'})

# Cruza o metadados com novos metadados provenientes das novas imagens
metadados2 = pd.merge(metadados,metadados_augmentedDataset,left_on=['arquivo'],right_on=['arquivo'])
metadados2 = pd.merge(metadados2,metadados_normalizedDataset,left_on=['convertida'],right_on=['convertida'])
metadados2 = pd.merge(metadados2,metadados_originalGrayDataset,left_on=['arquivo'],right_on=['arquivo'])

In [ ]:
metadados2.head()

,arquivo,classe,objecto,repeticao,fundo,iluminacao,local,resolucao,tamanho,convertida,normalizada,gray
0,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0,20210417_153013_gray.jpg,20210417_153013_gray_equ.jpg,20210417_153013_gray2.jpg
1,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0,20210417_153013_grad.jpg,20210417_153013_grad_equ.jpg,20210417_153013_gray2.jpg
2,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0,20210417_153013_excgrad.jpg,20210417_153013_excgrad_equ.jpg,20210417_153013_gray2.jpg
3,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0,20210417_153013_log.jpg,20210417_153013_log_equ.jpg,20210417_153013_gray2.jpg
4,20210417_153013.jpg,óculos,óculos gucci,1,branco_2,dia,interno,2560x1440,1144814.0,20210417_153013_exp.jpg,20210417_153013_exp_equ.jpg,20210417_153013_gray2.jpg


## Plota imagens filtradas (nome_arquivo_filtro.jpg) e normalizadas pelo equalizador (nome_arquivo_filtro_equ.jpg)
legenda filtro: \
gray (augmentedDataSet): níveis de cinza \
gray2 (originalGrayDataset): níveis de cinza \
grad (augmentedDataSet): gradiente \
exgrad (augmentedDataSet): soma de fundo com gradiente \
log (augmentedDataSet): logaritmo \
exp (augmentedDataSet): exponencial \
equ (normalizedDataset): normalizada pelo equalizador \

In [ ]:
# Plota imagens conforme classe desejada
plot_img_filtered(run=True)

Output hidden; open in https://colab.research.google.com to view.